![NYC Skyline](nyc.jpg)

Bienvenue à New York, l’une des villes les plus visitées au monde. Il y a de nombreuses annonces Airbnb à New York pour répondre à la forte demande de logements temporaires pour les voyageurs, qui peuvent aller de quelques nuits à plusieurs mois. Dans ce projet, nous allons examiner de plus près le marché Airbnb de New York en combinant des données provenant de plusieurs types de fichiers comme .csv, .tsv et .xlsx.

Rappelons que les fichiers CSV, TSV et Excel sont trois formats courants pour stocker des données. Trois fichiers contenant des données sur les annonces Airbnb de 2019 sont disponibles :

**data/airbnb_price.csv**
Ceci est un fichier CSV contenant des données sur les prix et les emplacements des annonces Airbnb.
- **`listing_id`**: identifiant unique de l’annonce
- **`price`**: prix de la nuitée en USD
- **`nbhood_full`**: nom de l’arrondissement et du quartier où se trouve l’annonce

**data/airbnb_room_type.xlsx**
Ceci est un fichier Excel contenant des données sur les descriptions et les types de chambres des annonces Airbnb.
- **`listing_id`**: identifiant unique de l’annonce
- **`description`**: description de l’annonce
- **`room_type`**: Airbnb propose trois types de chambres : chambres partagées, chambres privées et logements/appartements entiers

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: identifiant unique de l’annonce
- **`host_name`**: nom de l’hôte de l’annonce
- **`last_review`**: date de la dernière évaluation de l’annonce

In [4]:
# Import necessary packages  
import pandas as pd  
import numpy as np  
import datetime as dt  
import recordlinkage

# ----- 1. Chargement des données -----

# Importer le fichier CSV  
df_airbnb = "data/airbnb_price.csv"
df_price = pd.read_csv(df_airbnb)
print("Aperçu des données sur les prix:")
print(df_price.head())

Aperçu des données sur les prix:
   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown


In [5]:
# Importer le fichier TSV (Tab-separated values)
df_review = "data/airbnb_last_review.tsv"
review = pd.read_csv(df_review, sep='\t')
print("\nAperçu des données sur les évaluations:")
print(review.head())


Aperçu des données sur les évaluations:
   listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          Ben  June 09 2019


In [6]:
# Importer le fichier XLSX  
df_room_type = 'data/airbnb_room_type.xlsx'
xls = pd.ExcelFile(df_room_type)
print("\nFeuilles disponibles dans le fichier Excel:")
print(xls.sheet_names)  # Affiche le nom des feuilles  
room_df = xls.parse(0)
print("\nAperçu des données sur les types de chambre:")
print(room_df.head())


Feuilles disponibles dans le fichier Excel:
['airbnb_room_type']

Aperçu des données sur les types de chambre:
   listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt


In [7]:
# ----- 2. Nettoyage des données -----

# Nettoyage des données du fichier CSV  
print("\nInformations sur les données des prix:")
df_price.info()


Informations sur les données des prix:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


In [8]:
# Conversion de la colonne 'price' de l'objet en float  
df_price['price'] = df_price['price'].str.replace(' dollars', '')  # Retirer le texte ' dollars'
df_price['price'] = df_price['price'].astype('float')  # Conversion en float pour le calcul de la moyenne  
print("\nAperçu des données de prix après nettoyage:")
print(df_price.head())


Aperçu des données de prix après nettoyage:
   listing_id  price                nbhood_full
0        2595  225.0         Manhattan, Midtown
1        3831   89.0     Brooklyn, Clinton Hill
2        5099  200.0     Manhattan, Murray Hill
3        5178   79.0  Manhattan, Hell's Kitchen
4        5238  150.0       Manhattan, Chinatown


In [9]:
# Nettoyage des données du fichier TSV  
print("\nInformations sur les données des évaluations:")
print(review.info())


Informations sur les données des évaluations:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   host_name    25201 non-null  object
 2   last_review  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
None


In [10]:
# Conversion de la colonne 'last_review' en datetime  
review['last_review'] = pd.to_datetime(review['last_review'], infer_datetime_format=True, errors='coerce').dt.date  
print("\nAperçu des dates d'évaluation:")
print(review.head())


Aperçu des dates d'évaluation:
   listing_id    host_name last_review
0        2595     Jennifer  2019-05-21
1        3831  LisaRoxanne  2019-07-05
2        5099        Chris  2019-06-22
3        5178     Shunichi  2019-06-24
4        5238          Ben  2019-06-09


In [11]:
# Vérification des dates futures et correction  
today = dt.date.today()
review.loc[review['last_review'] > today, 'last_review'] = today  
print("\nDate la plus récente d'évaluation:")
print(review['last_review'].max())


Date la plus récente d'évaluation:
2019-07-09


In [12]:
# Nettoyage des données du fichier XLSX  
print("\nInformations sur les données des types de chambre:")
room_df.info()


Informations sur les données des types de chambre:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   description  25199 non-null  object
 2   room_type    25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


In [13]:
# Vérification des valeurs uniques dans la colonne 'room_type'
print("\nDistribution des types de chambre:")
print(room_df['room_type'].value_counts())


Distribution des types de chambre:
Entire home/apt    8458
Private room       7241
entire home/apt    2665
private room       2248
ENTIRE HOME/APT    2143
PRIVATE ROOM       1867
Shared room         380
shared room         110
SHARED ROOM          97
Name: room_type, dtype: int64


In [14]:
# Conversion de 'room_type' en catégories  
room_df['room_type'] = room_df['room_type'].astype('category')

# Mise à jour des catégories pour uniformiser les valeurs  
update_cat = {
    'ENTIRE HOME/APT': 'Entire Home/Apt',
    'entire home/apt': 'Entire Home/Apt',
    'Entire home/apt': 'Entire Home/Apt',
    'PRIVATE ROOM': 'Private Room',
    'Private room': 'Private Room',
    'private room': 'Private Room',
    'SHARED ROOM': 'Shared Room',
    'Shared room': 'Shared Room',
    'shared room': 'Shared Room',
}

# Remplacement des anciennes catégories par les nouvelles  
room_df['room_type'] = room_df['room_type'].replace(update_cat)

# Vérification de la mise à jour des types de chambre  
print("\nDistribution des types de chambre après mise à jour:")
print(room_df['room_type'].value_counts())


Distribution des types de chambre après mise à jour:
Entire Home/Apt    13266
Private Room       11356
Shared Room          587
Name: room_type, dtype: int64


In [15]:
# Reconvertir la colonne en catégorie  
room_df['room_type'] = room_df['room_type'].astype('category')
room_df.info()  # Vérification finale

# ----- 3. Vérification de la fusion des données -----

# Avant la jointure, vérifier que aucune donnée n'est perdue dans 'listing_id'
df_price['listing_id'] = df_price['listing_id'].astype('str')
review['listing_id'] = review['listing_id'].astype('str')
room_df['listing_id'] = room_df['listing_id'].astype('str')

# Initialisation de l'indexer pour le record linkage  
indexer = recordlinkage.Index()
indexer.block('listing_id')  # Génération des paires basées sur 'listing_id'
pairs = indexer.index(df_price, room_df)

# Création d'un objet de comparaison  
compare_cl = recordlinkage.Compare()
compare_cl.exact('listing_id', 'listing_id', label='listing_id')

# Calcul des correspondances potentielles  
potentiel_matche = compare_cl.compute(pairs, df_price, room_df)

# Vérification des objets qui ne correspondent pas  
print("\nCorrespondances potentielles sans perte de données:")
print(potentiel_matche[potentiel_matche.sum(axis=1) == 0])  # Vérification des paires sans correspondance

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   listing_id   25209 non-null  int64   
 1   description  25199 non-null  object  
 2   room_type    25209 non-null  category
dtypes: category(1), int64(1), object(1)
memory usage: 418.8+ KB

Correspondances potentielles sans perte de données:
Empty DataFrame
Columns: [listing_id]
Index: []
